**Solving tasks using one Conv2d(in_channels=10, out_channels=10, kernel_size=5, padding=2)

In [10]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import json
import os
from os.path import join as path_join

# Data Loading

In [11]:
def load_data(path):
    tasks = pd.Series()
    for file_path in os.listdir(path):
        task_file = path_join(path, file_path)

        with open(task_file, 'r') as f:
            task = json.load(f)

        tasks[file_path[:-5]] = task
    return tasks

In [13]:
train_tasks = load_data('C:/Users/Sai Karthik Yadav/ARC/data/training/')
evaluation_tasks = load_data('C:/Users/Sai Karthik Yadav/ARC/data/evaluation/')
test_tasks = load_data('C:/Users/Sai Karthik Yadav/ARC/data/test/')

train_tasks.head()

007bbfb7    {'test': [{'input': [[7, 0, 7], [7, 0, 7], [7,...
00d62c1b    {'train': [{'input': [[0, 0, 0, 0, 0, 0], [0, ...
017c7c7b    {'train': [{'input': [[0, 1, 0], [1, 1, 0], [0...
025d127b    {'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0,...
045e512c    {'train': [{'input': [[0, 0, 0, 0, 0, 0, 0, 0,...
dtype: object

# Model

In [18]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.nn import Conv2d
from torch import FloatTensor, LongTensor



def inp2img(inp):
    inp = np.array(inp)
    img = np.full((10, inp.shape[0], inp.shape[1]), 0, dtype=np.uint8)
    for i in range(10):
        img[i] = (inp==i)
    return img


class TaskSolver:        
    def train(self, task_train, n_epoch=3000):
        """basic pytorch train loop"""
        self.net = Conv2d(in_channels=10, out_channels=10, kernel_size=5, padding=2)
        
        criterion = CrossEntropyLoss()
        optimizer = Adam(self.net.parameters(), lr = 0.1)
        
        for epoch in range(n_epoch):
            for sample in task_train:
                inputs = FloatTensor(inp2img(sample['input'])).unsqueeze(dim=0)
                labels = LongTensor(sample['output']).unsqueeze(dim=0)
                
                optimizer.zero_grad()
                outputs = self.net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
        return self
            
    def predict(self, task_test):
        predictions = []
        with torch.no_grad():
            for sample in task_test:
                inputs = FloatTensor(inp2img(sample['input'])).unsqueeze(dim=0)
                outputs = self.net(inputs)
                pred =  outputs.squeeze(dim=0).cpu().numpy().argmax(0)
                predictions.append(pred)
                                     
        return predictions

# Results

In [19]:
def input_output_shape_is_same(task):
    return all([np.array(el['input']).shape == np.array(el['output']).shape for el in task['train']])

def calk_score(task_test, predict):
    return [int(np.equal(sample['output'], pred).all()) for sample, pred in zip(task_test, predict)]

In [20]:
def evaluate(tasks):
    ts = TaskSolver()
    result = []
    predictions = []
    for task in tqdm(tasks):
        if input_output_shape_is_same(task):
            ts.train(task['train'])
            pred = ts.predict(task['test'])
            score = calk_score(task['test'], pred)
        else:
            pred = [el['input'] for el in task['test']]
            score = [0]*len(task['test'])
        
        predictions.append(pred)
        result.append(score)
       
    return result, predictions

In [21]:
train_result, train_predictions = evaluate(train_tasks)
train_solved = [any(score) for score in train_result]

total = sum([len(score) for score in train_result])
print(f"solved : {sum(train_solved)} from {total} ({sum(train_solved)/total})")

100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [52:55<00:00,  7.94s/it]


solved : 19 from 416 (0.04567307692307692)


In [22]:
evaluation_result, evaluation_predictions = evaluate(evaluation_tasks)
evaluation_solved = [any(score) for score in evaluation_result]

total = sum([len(score) for score in evaluation_result])
print(f"solved : {sum(evaluation_solved)} from {total} ({sum(evaluation_solved)/total})")

100%|██████████████████████████████████████████████████████████████████████████████| 400/400 [1:14:40<00:00, 15.51s/it]


solved : 3 from 419 (0.007159904534606206)
